In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sb
import geopy
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
#import and read the csv
dfModel = pd.read_csv("Model_Data.csv")
dfPlot = pd.read_csv('Plot_Data.csv')
dfDesc = pd.read_csv('Fuel_Model_Descriptions.csv')
dfSoil = pd.read_csv('Soil_Moisture.csv')

#consturct the coordinates column from lat and lon columns and mapping these two columns as string
dfModel['latlng'] = dfModel.apply(lambda row: '{},{}'.format(row['lat'], row['lon']), axis=1)
dfModel

,Plot_ID2,Plot_ID_orig.x,dens_ALL,dens_ABGR,dens_LAOC,dens_PICO,dens_PIEN,dens_PIPO,dens_PSME,dens_ABAM,...,d2liveTree,d2Conif,d2ConifNotLP,shrb_pcov,shrb_avht,shrb_mxht,grs_pcov,grs_avht,grs_mxht,latlng
0,005560_CMS,005560_CMS,8593.750000,0.000000,468.750000,0.000000,1015.625000,0.000000,7031.250000,0.0000,...,2.743200,2.743200,2.743200,46.50,1.611111,1.958333,82.50,0.2500,0.4375,"48.60894432,-118.5411939"
1,008124_CHN,008124_CHN,1628.851371,0.000000,310.849498,584.397057,422.755318,0.000000,248.679599,62.1699,...,47.548798,47.548798,47.548798,57.50,1.472222,1.826389,0.75,0.2500,0.2500,"48.60190042,-118.4985233"
2,046653_CHS,046653_CHN,62.169900,12.433980,0.000000,0.000000,0.000000,24.867960,24.867960,0.0000,...,128.930396,128.930396,128.930396,17.50,0.250000,0.437500,13.75,0.2500,0.3750,"47.68883976,-120.4707228"
3,046788_CMN,046788_CMN,683.868896,360.585418,0.000000,24.867960,0.000000,87.037859,149.207759,62.1699,...,60.655198,60.655198,60.655198,26.25,1.236111,1.513889,0.50,0.8750,0.8750,"47.6868619,-120.4707829"
4,046968_CMN,046968_CMN,72812.500000,27500.000000,0.000000,937.500000,0.000000,1562.500000,10000.000000,0.0000,...,3.352800,3.352800,3.352800,64.25,1.909722,2.430556,0.25,0.2500,0.5000,"47.68567341,-120.4555554"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,999901_DMS,999901_DMN,4062.500000,0.000000,390.625000,0.000000,0.000000,0.000000,3671.875000,0.0000,...,0.914400,0.914400,0.914400,61.50,1.395833,1.666667,40.00,0.2500,0.2500,"48.61221144,-118.5589235"
244,999902_DMS,999902_DMS,19375.000000,0.000000,5312.500000,0.000000,1250.000000,0.000000,12812.500000,0.0000,...,3.048000,3.048000,6.096000,30.00,0.250000,0.625000,52.50,0.3125,1.0000,"48.69848776,-118.4997917"
245,999903_CMS,999903_CMS,6718.750000,0.000000,78.125000,78.125000,0.000000,0.000000,6562.500000,0.0000,...,8.229600,8.229600,8.229600,27.50,0.250000,0.437500,37.50,0.2500,0.3750,"48.70077402,-118.5064114"
246,999904_CMS,999904_CMS,10156.250000,0.000000,2187.500000,0.000000,703.125000,0.000000,7265.625000,0.0000,...,0.609600,0.609600,0.609600,50.00,0.312500,0.875000,18.75,0.1875,0.7500,"48.69657675,-118.5011834"


In [3]:
# Create a service provider locator this time pass timeout to be 10 mins because we have many rows. It will prevent timeout errors that you might encounter during the process
locator = Nominatim(user_agent="geoapiExercises")
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [4]:
# Using 0 raw of latitude and longitude from dmModel dataset for testing 
Latitude = "48.60894432"
Longitude = "-118.5411939"
  
location = locator.reverse(Latitude+","+Longitude)
  
location.raw

{'place_id': 222345312,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 584002466,
 'lat': '48.60935831215436',
 'lon': '-118.5408112946443',
 'display_name': 'State Highway 20, Ferry County, Washington, United States',
 'address': {'road': 'State Highway 20',
  'county': 'Ferry County',
  'state': 'Washington',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['48.5951748', '48.6299333', '-118.5793026', '-118.4111677']}

In [5]:
# Convert the data to a DataFrame
pd.json_normalize(location.raw['address'])

,road,county,state,country,country_code
0,State Highway 20,Ferry County,Washington,United States,us


In [6]:
def get_adress(x):
    coordinates = [x.lat,x.lon]
    loc = locator.reverse(coordinates)
    address = loc.raw
    address_coverted = pd.json_normalize(address).squeeze()
    return address_coverted

In [7]:
address_dfModel = dfModel.apply(get_adress, axis=1)
address_dfModel

,address.country,address.country_code,address.county,address.hamlet,address.house_number,address.postcode,address.road,address.state,boundingbox,display_name,lat,licence,lon,osm_id,osm_type,place_id
0,United States,us,Ferry County,NaN,NaN,NaN,State Highway 20,Washington,"[48.5951748, 48.6299333, -118.5793026, -118.41...","State Highway 20, Ferry County, Washington, Un...",48.60935831215436,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.5408112946443,584002466,way,222345312
1,United States,us,Ferry County,NaN,NaN,NaN,State Highway 20,Washington,"[48.5951748, 48.6299333, -118.5793026, -118.41...","State Highway 20, Ferry County, Washington, Un...",48.60427611434518,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.49884788385998,584002466,way,222345312
2,United States,us,Chelan County,NaN,NaN,NaN,NaN,Washington,"[47.2608714, 48.550725, -121.180713, -119.859708]","Chelan County, Washington, United States",47.8012197,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-120.6276904,1153517,relation,282552658
3,United States,us,Chelan County,NaN,NaN,NaN,NaN,Washington,"[47.2608714, 48.550725, -121.180713, -119.859708]","Chelan County, Washington, United States",47.8012197,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-120.6276904,1153517,relation,282552658
4,United States,us,Chelan County,NaN,NaN,NaN,NaN,Washington,"[47.2608714, 48.550725, -121.180713, -119.859708]","Chelan County, Washington, United States",47.8012197,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-120.6276904,1153517,relation,282552658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,United States,us,Ferry County,NaN,NaN,NaN,State Highway 20,Washington,"[48.5951748, 48.6299333, -118.5793026, -118.41...","State Highway 20, Ferry County, Washington, Un...",48.6138626690514,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.5591412519216,584002466,way,222345312
244,United States,us,Ferry County,NaN,NaN,NaN,National Forest Road 2140-250 / Marcus Road,Washington,"[48.6904963, 48.7005593, -118.535737, -118.501...","National Forest Road 2140-250 / Marcus Road, F...",48.6941141,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.5020326,5889910,way,97061498
245,United States,us,Ferry County,NaN,NaN,NaN,National Forest Road 2140-250 / Marcus Road,Washington,"[48.6904963, 48.7005593, -118.535737, -118.501...","National Forest Road 2140-250 / Marcus Road, F...",48.70050491778322,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.50634634138423,5889910,way,97061498
246,United States,us,Ferry County,NaN,NaN,NaN,National Forest Road 2140-250 / Marcus Road,Washington,"[48.6904963, 48.7005593, -118.535737, -118.501...","National Forest Road 2140-250 / Marcus Road, F...",48.69411808374264,"Data © OpenStreetMap contributors, ODbL 1.0. h...",-118.5020439814831,5889910,way,97061498


In [8]:
temp_address_dfModel = address_dfModel.filter(['address.county', 'address.state'])
temp_address_dfModel

,address.county,address.state
0,Ferry County,Washington
1,Ferry County,Washington
2,Chelan County,Washington
3,Chelan County,Washington
4,Chelan County,Washington
...,...,...
243,Ferry County,Washington
244,Ferry County,Washington
245,Ferry County,Washington
246,Ferry County,Washington


In [9]:
temp_dfModel = dfModel.filter(['Plot_ID_orig.x', 'FireSev', 'Salvage', 'lat', 'lon', 'nFires', 'elev', 'slope_albers'])
temp_dfModel

,Plot_ID_orig.x,FireSev,Salvage,lat,lon,nFires,elev,slope_albers
0,005560_CMS,M,S,48.608944,-118.541194,1,1348,12.260147
1,008124_CHN,H,N,48.601900,-118.498523,1,1697,20.097992
2,046653_CHN,H,S,47.688840,-120.470723,1,1390,25.850988
3,046788_CMN,M,N,47.686862,-120.470783,2,1335,32.534698
4,046968_CMN,M,N,47.685673,-120.455555,1,1054,19.188419
...,...,...,...,...,...,...,...,...
243,999901_DMN,M,S,48.612211,-118.558924,1,1293,13.206567
244,999902_DMS,M,S,48.698488,-118.499792,1,1594,16.419546
245,999903_CMS,M,S,48.700774,-118.506411,1,1484,17.676495
246,999904_CMS,M,S,48.696577,-118.501183,1,1530,17.932003


In [10]:
temp_dfModel = temp_dfModel.rename(columns = {'Plot_ID_orig.x': 'Plot_ID'})
temp_dfModel

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers
0,005560_CMS,M,S,48.608944,-118.541194,1,1348,12.260147
1,008124_CHN,H,N,48.601900,-118.498523,1,1697,20.097992
2,046653_CHN,H,S,47.688840,-120.470723,1,1390,25.850988
3,046788_CMN,M,N,47.686862,-120.470783,2,1335,32.534698
4,046968_CMN,M,N,47.685673,-120.455555,1,1054,19.188419
...,...,...,...,...,...,...,...,...
243,999901_DMN,M,S,48.612211,-118.558924,1,1293,13.206567
244,999902_DMS,M,S,48.698488,-118.499792,1,1594,16.419546
245,999903_CMS,M,S,48.700774,-118.506411,1,1484,17.676495
246,999904_CMS,M,S,48.696577,-118.501183,1,1530,17.932003


In [11]:
# Merge this address data back to the original DataFrame
dfModel_add = pd.concat([temp_dfModel, temp_address_dfModel], axis=1)
dfModel_add

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers,address.county,address.state
0,005560_CMS,M,S,48.608944,-118.541194,1,1348,12.260147,Ferry County,Washington
1,008124_CHN,H,N,48.601900,-118.498523,1,1697,20.097992,Ferry County,Washington
2,046653_CHN,H,S,47.688840,-120.470723,1,1390,25.850988,Chelan County,Washington
3,046788_CMN,M,N,47.686862,-120.470783,2,1335,32.534698,Chelan County,Washington
4,046968_CMN,M,N,47.685673,-120.455555,1,1054,19.188419,Chelan County,Washington
...,...,...,...,...,...,...,...,...,...,...
243,999901_DMN,M,S,48.612211,-118.558924,1,1293,13.206567,Ferry County,Washington
244,999902_DMS,M,S,48.698488,-118.499792,1,1594,16.419546,Ferry County,Washington
245,999903_CMS,M,S,48.700774,-118.506411,1,1484,17.676495,Ferry County,Washington
246,999904_CMS,M,S,48.696577,-118.501183,1,1530,17.932003,Ferry County,Washington


In [12]:
temp_dfPlot = dfPlot.filter(['Plot_ID', 'Date', 'Fuel_Model', 'Notes'])
temp_dfPlot

,Plot_ID,Date,Fuel_Model,Notes
0,159805_MHN,11/6/2017,GR2,Evidence of salvage logging. Plot should be re...
1,110508_DMN,10/4/2017,GS2,Plot found on popular hiking trail and moved 5...
2,110455_MMN,10/4/2017,TU2,All snags with snapped tops from windthrow. Do...
3,110098_DMN,10/4/2017,TU1,West quadrants are more open with mostly grass...
4,076213_DHS,9/20/2017,GS3,No evidence of salvage logging
...,...,...,...,...
270,78785_DMS,10/13/2017,NaN,Likely same PCT operation as 78853_CMS
271,78826_DMS,10/13/2017,NaN,Plot initially completed on new fire training ...
272,78853_CMS,10/13/2017,NaN,NaN
273,78898_DMS,10/13/2017,NaN,NaN


In [13]:
dfPM = pd.merge(dfModel_add, temp_dfPlot, on = 'Plot_ID', how="inner")
dfPM

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers,address.county,address.state,Date,Fuel_Model,Notes
0,005560_CMS,M,S,48.608944,-118.541194,1,1348,12.260147,Ferry County,Washington,10/26/2017,TU5/TU3,Very old road passes through plot. Many diseas...
1,008124_CHN,H,N,48.601900,-118.498523,1,1697,20.097992,Ferry County,Washington,10/26/2017,TL4,NaN
2,046653_CHN,H,S,47.688840,-120.470723,1,1390,25.850988,Chelan County,Washington,9/14/2017,TU1,Evidence of salvage logging in plot
3,046788_CMN,M,N,47.686862,-120.470783,2,1335,32.534698,Chelan County,Washington,9/14/2017,TU2,NaN
4,046968_CMN,M,N,47.685673,-120.455555,1,1054,19.188419,Chelan County,Washington,9/19/2017,TU5,Large DBH trees had recently fallen
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,999901_DMN,M,S,48.612211,-118.558924,1,1293,13.206567,Ferry County,Washington,10/13/2017,TU3,Evidence of salvage logging. Small WL seedling...
246,999902_DMS,M,S,48.698488,-118.499792,1,1594,16.419546,Ferry County,Washington,8/4/2017,TU1,Javad initially didn't work on 8/4/17. Returne...
247,999903_CMS,M,S,48.700774,-118.506411,1,1484,17.676495,Ferry County,Washington,8/4/2017,TU1,Heavily grazed by cattle. Primitive trail take...
248,999904_CMS,M,S,48.696577,-118.501183,1,1530,17.932003,Ferry County,Washington,8/4/2017,TL3,Javad initially didn't work on 8/4/17. Returne...


In [14]:
temp_dfDesc = dfDesc.filter(['FM Code', 'Fuel Model Name', 'Wind Adj', 'Moist Extinct', 'Dead Heat', 'Live Heat'])
temp_dfDesc = temp_dfDesc.rename(columns = {'FM Code': 'Fuel_Model'})
temp_dfDesc

,Fuel_Model,Fuel Model Name,Wind Adj,Moist Extinct,Dead Heat,Live Heat
0,FB1,Short Grass,0.36,12,8000,--
1,FB2,Timber grass and understory,0.36,15,8000,8000
2,GR1,"Short, sparse dry climate grass",0.31,15,8000,8000
3,GR2,Low load dry climate grass,0.36,15,8000,8000
4,GR4,Moderate load dry climate grass,0.42,15,8000,8000
5,GR7,High load dry climate grass,0.46,15,8000,8000
6,GS1,Low load dry climate grass-shrub,0.35,15,8000,8000
7,GS2,Moderate load dry climate grass-shrub,0.39,15,8000,8000
8,FB3,Tall grass,0.44,25,8000,--
9,GR3,Low load very coarse humid climate grass,0.42,30,8000,8000


In [15]:
dfPD = pd.merge(dfPM, temp_dfDesc, on = 'Fuel_Model', how="left")
dfPD

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers,address.county,address.state,Date,Fuel_Model,Notes,Fuel Model Name,Wind Adj,Moist Extinct,Dead Heat,Live Heat
0,005560_CMS,M,S,48.608944,-118.541194,1,1348,12.260147,Ferry County,Washington,10/26/2017,TU5/TU3,Very old road passes through plot. Many diseas...,NaN,NaN,NaN,NaN,NaN
1,008124_CHN,H,N,48.601900,-118.498523,1,1697,20.097992,Ferry County,Washington,10/26/2017,TL4,NaN,Small downed logs,0.31,25.0,8000.0,--
2,046653_CHN,H,S,47.688840,-120.470723,1,1390,25.850988,Chelan County,Washington,9/14/2017,TU1,Evidence of salvage logging in plot,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000
3,046788_CMN,M,N,47.686862,-120.470783,2,1335,32.534698,Chelan County,Washington,9/14/2017,TU2,NaN,Moderate load humid climate timber-shrub,0.36,30.0,8000.0,8000
4,046968_CMN,M,N,47.685673,-120.455555,1,1054,19.188419,Chelan County,Washington,9/19/2017,TU5,Large DBH trees had recently fallen,Very high load dry climate timber-shrub,0.33,25.0,8000.0,8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,999901_DMN,M,S,48.612211,-118.558924,1,1293,13.206567,Ferry County,Washington,10/13/2017,TU3,Evidence of salvage logging. Small WL seedling...,Moderate load humid climate timber-grass-shrub,0.38,30.0,8000.0,8000
246,999902_DMS,M,S,48.698488,-118.499792,1,1594,16.419546,Ferry County,Washington,8/4/2017,TU1,Javad initially didn't work on 8/4/17. Returne...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000
247,999903_CMS,M,S,48.700774,-118.506411,1,1484,17.676495,Ferry County,Washington,8/4/2017,TU1,Heavily grazed by cattle. Primitive trail take...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000
248,999904_CMS,M,S,48.696577,-118.501183,1,1530,17.932003,Ferry County,Washington,8/4/2017,TL3,Javad initially didn't work on 8/4/17. Returne...,Moderate load conifer litter,0.29,20.0,8000.0,--


In [16]:
temp_dfSoil = dfSoil.filter(['Name', 'State', 'County', 'Latitude', 'Longitude', 'Elevation_feet'])

# Convert elevation feet to meters in integer
temp_dfSoil = temp_dfSoil.assign(Elevation_m = lambda x: x['Elevation_feet'] * 0.3084)
temp_dfSoil['Elevation_m'] = round(temp_dfSoil['Elevation_m'],0).astype(int)
temp_dfSoil

,Name,State,County,Latitude,Longitude,Elevation_feet,Elevation_m
0,Adamsville,DE,Kent,38.83300,-75.68300,43,13
1,Atglen,DE,Chester,39.91700,-75.98300,663,204
2,Bethany_Beach,DE,Sussex,38.55000,-75.06700,7,2
3,Blackbird,DE,New_Castle,39.38300,-75.63300,41,13
4,Blackiston,DE,Kent,39.23300,-75.73300,56,17
...,...,...,...,...,...,...,...
2101,TxSON 29,TX,NaN,30.25340,-98.69903,1527,471
2102,TxSON 30,TX,NaN,30.24921,-98.69950,1522,469
2103,TxSON 31,TX,NaN,30.41932,-98.80461,1795,554
2104,TxSON 32,TX,NaN,30.46132,-98.84506,1966,606


In [17]:
temp_dfSoil = temp_dfSoil.rename(columns = {'Elevation_m': 'elev'})
temp_dfSoil.drop('Elevation_feet', axis=1, inplace=True)

In [18]:
finalmg = pd.merge(dfPD, temp_dfSoil, on = 'elev', how="outer")
finalmg

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers,address.county,address.state,...,Fuel Model Name,Wind Adj,Moist Extinct,Dead Heat,Live Heat,Name,State,County,Latitude,Longitude
0,005560_CMS,M,S,48.608944,-118.541194,1.0,1348,12.260147,Ferry County,Washington,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84386_CHS,H,S,48.712194,-118.525829,1.0,1348,29.917093,Ferry County,Washington,...,Very high load dry climate timber-shrub,0.33,25.0,8000.0,8000,NaN,NaN,NaN,NaN,NaN
2,008124_CHN,H,N,48.601900,-118.498523,1.0,1697,20.097992,Ferry County,Washington,...,Small downed logs,0.31,25.0,8000.0,--,NaN,NaN,NaN,NaN,NaN
3,046653_CHN,H,S,47.688840,-120.470723,1.0,1390,25.850988,Chelan County,Washington,...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000,Jordan_Valley_Cwma,ID,Owyhee,42.95000,-117.01000
4,046653_CHN,H,S,47.688840,-120.470723,1.0,1390,25.850988,Chelan County,Washington,...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000,LuckyHills,AZ,Cochise,31.74000,-110.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,NaN,NaN,NaN,NaN,NaN,NaN,536,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 9,TX,NaN,30.42833,-98.80651
2286,NaN,NaN,NaN,NaN,NaN,NaN,465,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 24,TX,NaN,30.24556,-98.69876
2287,NaN,NaN,NaN,NaN,NaN,NaN,558,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 27,TX,NaN,30.43154,-98.86041
2288,NaN,NaN,NaN,NaN,NaN,NaN,469,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 30,TX,NaN,30.24921,-98.69950


In [19]:
finalmg = finalmg.dropna(subset=['lat', 'lon', 'Latitude', 'Longitude'], how='all')
finalmg

,Plot_ID,FireSev,Salvage,lat,lon,nFires,elev,slope_albers,address.county,address.state,...,Fuel Model Name,Wind Adj,Moist Extinct,Dead Heat,Live Heat,Name,State,County,Latitude,Longitude
0,005560_CMS,M,S,48.608944,-118.541194,1.0,1348,12.260147,Ferry County,Washington,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84386_CHS,H,S,48.712194,-118.525829,1.0,1348,29.917093,Ferry County,Washington,...,Very high load dry climate timber-shrub,0.33,25.0,8000.0,8000,NaN,NaN,NaN,NaN,NaN
2,008124_CHN,H,N,48.601900,-118.498523,1.0,1697,20.097992,Ferry County,Washington,...,Small downed logs,0.31,25.0,8000.0,--,NaN,NaN,NaN,NaN,NaN
3,046653_CHN,H,S,47.688840,-120.470723,1.0,1390,25.850988,Chelan County,Washington,...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000,Jordan_Valley_Cwma,ID,Owyhee,42.95000,-117.01000
4,046653_CHN,H,S,47.688840,-120.470723,1.0,1390,25.850988,Chelan County,Washington,...,Light load dry climate timber-grass-shrub,0.33,20.0,8000.0,8000,LuckyHills,AZ,Cochise,31.74000,-110.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285,NaN,NaN,NaN,NaN,NaN,NaN,536,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 9,TX,NaN,30.42833,-98.80651
2286,NaN,NaN,NaN,NaN,NaN,NaN,465,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 24,TX,NaN,30.24556,-98.69876
2287,NaN,NaN,NaN,NaN,NaN,NaN,558,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 27,TX,NaN,30.43154,-98.86041
2288,NaN,NaN,NaN,NaN,NaN,NaN,469,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,TxSON 30,TX,NaN,30.24921,-98.69950


In [20]:
finalmg.describe()

,lat,lon,nFires,elev,slope_albers,Wind Adj,Moist Extinct,Dead Heat,Latitude,Longitude
count,275.000000,275.000000,275.000000,2290.000000,275.000000,255.000000,255.000000,255.0,2112.000000,2112.000000
mean,48.375088,-119.329412,1.010909,901.468996,20.609296,0.357922,25.431373,8000.0,39.184788,-103.595800
std,0.584160,0.999074,0.104065,918.312118,7.964068,0.049992,6.874164,0.0,6.694647,18.124457
min,46.523005,-120.990386,1.000000,-24.000000,2.052775,0.290000,15.000000,8000.0,10.280000,-170.210000
25%,47.687851,-120.423493,1.000000,180.000000,14.203597,0.330000,20.000000,8000.0,35.277500,-115.442500
50%,48.698488,-118.528882,1.000000,505.500000,20.179287,0.330000,25.000000,8000.0,38.540000,-102.625000
75%,48.950209,-118.411100,1.000000,1436.750000,26.059031,0.380000,30.000000,8000.0,42.020000,-94.757500
max,48.993247,-118.354262,2.000000,3725.000000,39.469452,0.550000,40.000000,8000.0,71.580000,128.910000
